In [8]:
import requests
import json
import pandas as pd
import datetime as dt

import warnings
warnings.filterwarnings('ignore')

In [229]:
def queryResultToDataframe(response) :
    json_object = json.loads(response.text)
    event = json_object['epcisBody']['queryResults']['resultBody']['eventList']

    df = pd.DataFrame()

    for e in event:
        row_data = {
            'eventTime': e['eventTime'],
            'eventTimeZoneOffset': e['eventTimeZoneOffset'],
            'type': e['type'],
            'action': e['action'],
            'bizStep': e['bizStep'],
            'disposition': e.get('disposition'), 
            'epcList': e['epcList'][0],
            'readPoint': e['readPoint']['id'],
            'bizLocation': e['bizLocation']['id'],
            'sourceList_type': e['sourceList'][0]['type'] if 'sourceList' in e and e['sourceList'] and 'type' in e['sourceList'][0] else None,
            'sourceList_source': e['sourceList'][0]['source'] if 'sourceList' in e and e['sourceList'] and 'source' in e['sourceList'][0] else None,
            'destinationList_type': e['destinationList'][0]['type'] if 'destinationList' in e and e['destinationList'] and 'type' in e['destinationList'][0] else None,
            'destinationList_source': e['destinationList'][0]['destination'] if 'destinationList' in e and e['destinationList'] and 'destination' in e['destinationList'][0] else None,
        }
        df = df.append(row_data, ignore_index=True)

    return df

In [230]:
def queryResultToDataframe_MRO(response) :
    json_object = json.loads(response.text)
    event = json_object['epcisBody']['queryResults']['resultBody']['eventList']

    df = pd.DataFrame()

    for e in event:
        row_data = {
            'eventTime': e['eventTime'],
            'eventTimeZoneOffset': e['eventTimeZoneOffset'],
            'type': e['type'],
            'action': e['action'],
            'bizStep': e['bizStep'],
            'disposition': e.get('disposition'), 
            'epcList': e['epcList'][0],
            'readPoint': e['readPoint']['id'],
            'bizLocation': e['bizLocation']['id'],
            'MRO:company': e.get('MRO:company'),
            'MRO:manager': e.get('MRO:manager'),
            'MRO:contact': e.get('MRO:contact'),
            'MRO:contents': e.get('MRO:contents')
        }
        df = df.append(row_data, ignore_index=True)

    return df

In [19]:
# 스캔을 통해 digital link 획득
# url = "http://localhost:8091/epcis/v2/bizLocations/https%3A%2F%2Fid.oliot.org%2F414%2F880123451006/events"

query_json = {
  "name": "status",
  "query": {
    "eventType": [
        "ObjectEvent"
    ],
    "EQ_bizStep": [
      "accepting"  
    ]
  }
}

response = requests.post("http://localhost:8091/epcis/v2/queries", json=query_json) 
response

# 여기서 스캔통해 얻은 링크 검색 > 해당 이벤트의 상태시간, 현재위치 가져오기
# 이때 status는 자산 조회 후 모두 최신거 가져오도록 로직 필요함

<Response [201]>

In [231]:
status_events = requests.get("http://localhost:8091/epcis/v2/queries/status/events", verify=False)  # 쿼리로 이벤트 가져오기

# df = queryResultToDataframe(status_events)
df = queryResultToDataframe(status_events)
df

,eventTime,eventTimeZoneOffset,type,action,bizStep,disposition,epcList,readPoint,bizLocation,sourceList_type,sourceList_source,destinationList_type,destinationList_source
0,2023-08-04 10:28:30,+09:00,ObjectEvent,OBSERVE,accepting,None,https://id.oliot.org/8004/8801234561000,https://id.oliot.org/414/880123451006/254/20201,https://id.oliot.org/414/880123451006,location,https://id.oliot.org/414/880123451006/254/20201,location,https://id.oliot.org/414/880123451006/254/20203
1,2023-08-04 09:00:45,+09:00,ObjectEvent,OBSERVE,accepting,active,https://id.oliot.org/8004/8801234561212,https://id.oliot.org/414/880123451006/254/20203,https://id.oliot.org/414/880123451006,None,None,None,None
2,2023-08-04 09:00:30,+09:00,ObjectEvent,OBSERVE,accepting,active,https://id.oliot.org/8004/8801234561000,https://id.oliot.org/414/880123451006/254/20201,https://id.oliot.org/414/880123451006,None,None,None,None
3,2023-08-04 09:00:28,+09:00,ObjectEvent,OBSERVE,accepting,available,https://id.oliot.org/8004/8801234561000,https://id.oliot.org/414/880123451006/254/20201,https://id.oliot.org/414/880123451006,None,None,None,None
4,2023-08-02 17:15:51,+09:00,ObjectEvent,OBSERVE,accepting,active,https://id.oliot.org/8004/8801234562222,https://id.oliot.org/414/880123451006/254/20203,https://id.oliot.org/414/880123451006,None,None,None,None
5,2023-08-02 17:15:37,+09:00,ObjectEvent,OBSERVE,accepting,available,https://id.oliot.org/8004/8801234561111,https://id.oliot.org/414/880123451006/254/20203,https://id.oliot.org/414/880123451006,None,None,None,None
6,2023-08-02 17:15:12,+09:00,ObjectEvent,OBSERVE,accepting,None,https://id.oliot.org/8004/8801234562222,https://id.oliot.org/414/880123451006/254/30301,https://id.oliot.org/414/880123451006,location,https://id.oliot.org/414/880123451006/254/30301,location,https://id.oliot.org/414/880123451006/254/20203
7,2023-08-02 17:14:49,+09:00,ObjectEvent,OBSERVE,accepting,None,https://id.oliot.org/8004/8801234561111,https://id.oliot.org/414/880123451006/254/20201,https://id.oliot.org/414/880123451006,location,https://id.oliot.org/414/880123451006/254/20201,location,https://id.oliot.org/414/880123451006/254/20203


In [4]:
# # pip install pyOpenSSL
# import ssl
# import urllib3
# print(ssl.OPENSSL_VERSION)

In [232]:
q_epcs = requests.get("http://localhost:8091/epcis/v2/epcs", verify=False)  # 쿼리로 이벤트 가져오기
epcs_json = json.loads(q_epcs.text)
epcs = epcs_json['member']

df_status = pd.DataFrame()
df_location_status = pd.DataFrame()


for epc in epcs:
  df_epc = df[df['epcList'] == epc]

  diposition_info = df_epc[df_epc['disposition'].notnull()]
  diposition_info.sort_values('eventTime', axis=0, ascending=False, inplace=True)
  diposition_info.reset_index(drop=True, inplace=True)

  source_info = df_epc[df_epc['disposition'].isnull()]
  source_info.sort_values('eventTime', axis=0, ascending=False, inplace=True)
  source_info.reset_index(drop=True, inplace=True)

  # if not source_info.empty:
  #   e_time = source_info["eventTime"][0]
  #   n_location = source_info["destinationList_source"][0]
  #   p_location = source_info["sourceList_source"][0]
  # else:
  #   e_time = diposition_info["eventTime"][0]
  #   status = diposition_info["disposition"][0]
  #   n_location = diposition_info["readPoint"][0] # 자산이동이 없는 경우
  #   p_location = None

  row_status = {
    "상태변경일": e_time,
    "자산": epc,
    "상태": status,
    "현재위치": n_location,
  }

  row_location_status = {
    "위치변경일": e_time,
    "자산": epc,
    "현재위치": n_location,
    "지난위치": p_location,
  }


  df_status = df_status.append(row_status, ignore_index=True)
  df_location_status = df_location_status.append(row_location_status, ignore_index=True)

df_status

KeyError: 0

In [180]:
df_location_status

,위치변경일,자산,현재위치,지난위치
0,2023-08-04 09:00:45,https://id.oliot.org/8004/8801234561212,https://id.oliot.org/414/880123451006/254/20203,None
1,2023-08-04 10:28:30,https://id.oliot.org/8004/8801234561000,https://id.oliot.org/414/880123451006/254/20203,https://id.oliot.org/414/880123451006/254/20201
2,2023-08-02 17:15:12,https://id.oliot.org/8004/8801234562222,https://id.oliot.org/414/880123451006/254/20203,https://id.oliot.org/414/880123451006/254/30301
3,2023-08-02 17:14:49,https://id.oliot.org/8004/8801234561111,https://id.oliot.org/414/880123451006/254/20203,https://id.oliot.org/414/880123451006/254/20201


---

In [170]:
query_json = {
  "name": "MRO",
  "query": {
    "eventType": [
        "ObjectEvent"
    ],
    "EQ_bizStep": [
      "inspecting",
      "replacing",
      "repairing",
      "destroying"
    ]
  }
}

response = requests.post("http://localhost:8091/epcis/v2/queries", json=query_json) 
response

<Response [201]>

In [218]:
mro_events = requests.get("http://localhost:8091/epcis/v2/queries/MRO/events", verify=False) 
df = queryResultToDataframe_MRO(mro_events)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   eventTime            11 non-null     object
 1   eventTimeZoneOffset  11 non-null     object
 2   type                 11 non-null     object
 3   action               11 non-null     object
 4   bizStep              11 non-null     object
 5   disposition          10 non-null     object
 6   epcList              11 non-null     object
 7   readPoint            11 non-null     object
 8   bizLocation          11 non-null     object
 9   MRO:company          8 non-null      object
 10  MRO:manager          8 non-null      object
 11  MRO:contact          8 non-null      object
 12  MRO:contents         8 non-null      object
dtypes: object(13)
memory usage: 1.2+ KB


In [220]:
q_epcs = requests.get("http://localhost:8091/epcis/v2/epcs")  
epcs_json = json.loads(q_epcs.text)
epcs = epcs_json['member']

data = pd.DataFrame()

for epc in epcs:
  try:
    df_epc = df[df['epcList'] == epc]
    df_epc.sort_values('eventTime', axis=0, ascending=False, inplace=True)
    df_epc.reset_index(drop=True, inplace=True)

    for i in range(len(df_epc)):
      row_data = {
        "현재위치": df_epc["readPoint"][i],
        "자산": epc,
        "유지보수일": df_epc["eventTime"][i],
        "점검내용": df_epc["bizStep"][i],
        "유지보수내용": df_epc["disposition"][i],
        "관리업체": df_epc["MRO:company"][i],
        "담당자": df_epc["MRO:manager"][i],
        "담당연락처": df_epc["MRO:contact"][i],
        "기타": df_epc["MRO:contents"][i]
      }

      data = data.append(row_data, ignore_index=True) 

  except KeyError:
    pass

# data

In [223]:
unique_assets = df["epcList"].unique()

array(['https://id.oliot.org/8004/880123456112312',
       'https://id.oliot.org/8004/88012345611234',
       'https://id.oliot.org/8004/8801234561000'], dtype=object)